In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DateChallenge").getOrCreate()

data = [
    ("T001", "2025-08-28 14:30:00", 120.5),
    ("T002", "2024-12-31 23:59:59", 250.0),
    ("T003", "2023-01-15 08:15:45", 75.2),
    ("T004", "2022-06-05 18:05:30", 560.1),
    ("T005", None, 300.0)
]

transactions = spark.createDataFrame(data, ["TransactionID", "TransactionDate", "Amount"])
transactions.show(truncate=False)


25/08/29 10:52:51 WARN Utils: Your hostname, EmmanuelPC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/29 10:52:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/29 10:52:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+-------------+-------------------+------+
|TransactionID|TransactionDate    |Amount|
+-------------+-------------------+------+
|T001         |2025-08-28 14:30:00|120.5 |
|T002         |2024-12-31 23:59:59|250.0 |
|T003         |2023-01-15 08:15:45|75.2  |
|T004         |2022-06-05 18:05:30|560.1 |
|T005         |NULL               |300.0 |
+-------------+-------------------+------+



In [2]:
"""
Convertis la colonne TransactionDate (string) en timestamp.
Crée une colonne DateOnly contenant uniquement la date (yyyy-MM-dd).
Crée une colonne YearMonth au format yyyy-MM pour grouper par mois.
"""

'\nConvertis la colonne TransactionDate (string) en timestamp.\nCrée une colonne DateOnly contenant uniquement la date (yyyy-MM-dd).\nCrée une colonne YearMonth au format yyyy-MM pour grouper par mois.\n'

In [3]:
import pyspark.sql.functions as sf

In [4]:
transactions.select(sf.to_timestamp('TransactionDate')).printSchema()

root
 |-- to_timestamp(TransactionDate): timestamp (nullable = true)



In [5]:
transactions = transactions.withColumn( 'TransactionDate', sf.to_timestamp('TransactionDate'))

In [6]:
transactions.printSchema()

root
 |-- TransactionID: string (nullable = true)
 |-- TransactionDate: timestamp (nullable = true)
 |-- Amount: double (nullable = true)



In [7]:
transactions.withColumn('DateOnly' , sf.to_date('TransactionDate')).show()

+-------------+-------------------+------+----------+
|TransactionID|    TransactionDate|Amount|  DateOnly|
+-------------+-------------------+------+----------+
|         T001|2025-08-28 14:30:00| 120.5|2025-08-28|
|         T002|2024-12-31 23:59:59| 250.0|2024-12-31|
|         T003|2023-01-15 08:15:45|  75.2|2023-01-15|
|         T004|2022-06-05 18:05:30| 560.1|2022-06-05|
|         T005|               NULL| 300.0|      NULL|
+-------------+-------------------+------+----------+



In [8]:
transactions = transactions.withColumn('DateOnly' , sf.to_date('TransactionDate'))

In [9]:
transactions.printSchema()

root
 |-- TransactionID: string (nullable = true)
 |-- TransactionDate: timestamp (nullable = true)
 |-- Amount: double (nullable = true)
 |-- DateOnly: date (nullable = true)



In [10]:
transactions.withColumn('YearMonth' , sf.date_format('DateOnly' , 'yyyy-MM')).show()

+-------------+-------------------+------+----------+---------+
|TransactionID|    TransactionDate|Amount|  DateOnly|YearMonth|
+-------------+-------------------+------+----------+---------+
|         T001|2025-08-28 14:30:00| 120.5|2025-08-28|  2025-08|
|         T002|2024-12-31 23:59:59| 250.0|2024-12-31|  2024-12|
|         T003|2023-01-15 08:15:45|  75.2|2023-01-15|  2023-01|
|         T004|2022-06-05 18:05:30| 560.1|2022-06-05|  2022-06|
|         T005|               NULL| 300.0|      NULL|     NULL|
+-------------+-------------------+------+----------+---------+



In [11]:
transactions = transactions.withColumn('YearMonth' , sf.date_format('DateOnly' , 'yyyy-MM'))

In [12]:
"""
Extrais l’année, le mois, le jour, l’heure, les minutes et les secondes.
Crée une colonne DayOfWeek qui renvoie le jour de la semaine.
Crée une colonne WeekOfYear (1 → 52).
"""

'\nExtrais l’année, le mois, le jour, l’heure, les minutes et les secondes.\nCrée une colonne DayOfWeek qui renvoie le jour de la semaine.\nCrée une colonne WeekOfYear (1 → 52).\n'

In [13]:
transactions \
    .withColumn('year' , sf.year('TransactionDate')) \
    .withColumn('month' , sf.month('TransactionDate')) \
    .withColumn('day' , sf.day('TransactionDate')) \
    .withColumn('hour' , sf.hour('TransactionDate')) \
    .withColumn('minutes' , sf.minute('TransactionDate')) \
    .withColumn('seconds' , sf.second('TransactionDate')) \
.show()

+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+
|TransactionID|    TransactionDate|Amount|  DateOnly|YearMonth|year|month| day|hour|minutes|seconds|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+
|         T001|2025-08-28 14:30:00| 120.5|2025-08-28|  2025-08|2025|    8|  28|  14|     30|      0|
|         T002|2024-12-31 23:59:59| 250.0|2024-12-31|  2024-12|2024|   12|  31|  23|     59|     59|
|         T003|2023-01-15 08:15:45|  75.2|2023-01-15|  2023-01|2023|    1|  15|   8|     15|     45|
|         T004|2022-06-05 18:05:30| 560.1|2022-06-05|  2022-06|2022|    6|   5|  18|      5|     30|
|         T005|               NULL| 300.0|      NULL|     NULL|NULL| NULL|NULL|NULL|   NULL|   NULL|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+



In [14]:
transactions = transactions \
    .withColumn('year' , sf.year('TransactionDate')) \
    .withColumn('month' , sf.month('TransactionDate')) \
    .withColumn('day' , sf.day('TransactionDate')) \
    .withColumn('hour' , sf.hour('TransactionDate')) \
    .withColumn('minutes' , sf.minute('TransactionDate')) \
    .withColumn('seconds' , sf.second('TransactionDate')) 

In [15]:
transactions.withColumn('dayofweek' , sf.dayofweek('TransactionDate')).show()

+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+
|TransactionID|    TransactionDate|Amount|  DateOnly|YearMonth|year|month| day|hour|minutes|seconds|dayofweek|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+
|         T001|2025-08-28 14:30:00| 120.5|2025-08-28|  2025-08|2025|    8|  28|  14|     30|      0|        5|
|         T002|2024-12-31 23:59:59| 250.0|2024-12-31|  2024-12|2024|   12|  31|  23|     59|     59|        3|
|         T003|2023-01-15 08:15:45|  75.2|2023-01-15|  2023-01|2023|    1|  15|   8|     15|     45|        1|
|         T004|2022-06-05 18:05:30| 560.1|2022-06-05|  2022-06|2022|    6|   5|  18|      5|     30|        1|
|         T005|               NULL| 300.0|      NULL|     NULL|NULL| NULL|NULL|NULL|   NULL|   NULL|     NULL|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+



In [16]:
transactions = transactions.withColumn('dayofweek' , sf.dayofweek('TransactionDate'))

In [17]:
transactions = transactions.withColumn('weekofyear' , sf.weekofyear('TransactionDate'))

In [18]:
transactions.show()

+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+
|TransactionID|    TransactionDate|Amount|  DateOnly|YearMonth|year|month| day|hour|minutes|seconds|dayofweek|weekofyear|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+
|         T001|2025-08-28 14:30:00| 120.5|2025-08-28|  2025-08|2025|    8|  28|  14|     30|      0|        5|        35|
|         T002|2024-12-31 23:59:59| 250.0|2024-12-31|  2024-12|2024|   12|  31|  23|     59|     59|        3|         1|
|         T003|2023-01-15 08:15:45|  75.2|2023-01-15|  2023-01|2023|    1|  15|   8|     15|     45|        1|         2|
|         T004|2022-06-05 18:05:30| 560.1|2022-06-05|  2022-06|2022|    6|   5|  18|      5|     30|        1|        22|
|         T005|               NULL| 300.0|      NULL|     NULL|NULL| NULL|NULL|NULL|   NULL|   NULL|     NULL|      NULL|
+-------------+---------

In [19]:
"""
Ajoute une colonne UnixSeconds contenant le nombre de secondes depuis 1970-01-01.
Crée une colonne UnixMillis avec le nombre de millisecondes.
À partir de UnixSeconds, recrée une colonne BackToDate pour vérifier la conversion inverse.
Calcule le nombre de jours écoulés depuis la date de transaction jusqu’à aujourd’hui.
Calcule le nombre de secondes écoulées entre chaque transaction et 2025-01-01 00:00:00.
"""


'\nAjoute une colonne UnixSeconds contenant le nombre de secondes depuis 1970-01-01.\nCrée une colonne UnixMillis avec le nombre de millisecondes.\nÀ partir de UnixSeconds, recrée une colonne BackToDate pour vérifier la conversion inverse.\nCalcule le nombre de jours écoulés depuis la date de transaction jusqu’à aujourd’hui.\nCalcule le nombre de secondes écoulées entre chaque transaction et 2025-01-01 00:00:00.\n'

In [20]:
transactions = transactions.withColumn('UnixSeconds' , sf.unix_seconds('TransactionDate'))

In [21]:
transactions.show()

+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+
|TransactionID|    TransactionDate|Amount|  DateOnly|YearMonth|year|month| day|hour|minutes|seconds|dayofweek|weekofyear|UnixSeconds|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+
|         T001|2025-08-28 14:30:00| 120.5|2025-08-28|  2025-08|2025|    8|  28|  14|     30|      0|        5|        35| 1756384200|
|         T002|2024-12-31 23:59:59| 250.0|2024-12-31|  2024-12|2024|   12|  31|  23|     59|     59|        3|         1| 1735685999|
|         T003|2023-01-15 08:15:45|  75.2|2023-01-15|  2023-01|2023|    1|  15|   8|     15|     45|        1|         2| 1673766945|
|         T004|2022-06-05 18:05:30| 560.1|2022-06-05|  2022-06|2022|    6|   5|  18|      5|     30|        1|        22| 1654445130|
|         T005|               NULL| 300.0|      NULL|     NULL

In [22]:
transactions.withColumn('unix_millis' , sf.unix_millis('TransactionDate')).show()

+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+
|TransactionID|    TransactionDate|Amount|  DateOnly|YearMonth|year|month| day|hour|minutes|seconds|dayofweek|weekofyear|UnixSeconds|  unix_millis|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+
|         T001|2025-08-28 14:30:00| 120.5|2025-08-28|  2025-08|2025|    8|  28|  14|     30|      0|        5|        35| 1756384200|1756384200000|
|         T002|2024-12-31 23:59:59| 250.0|2024-12-31|  2024-12|2024|   12|  31|  23|     59|     59|        3|         1| 1735685999|1735685999000|
|         T003|2023-01-15 08:15:45|  75.2|2023-01-15|  2023-01|2023|    1|  15|   8|     15|     45|        1|         2| 1673766945|1673766945000|
|         T004|2022-06-05 18:05:30| 560.1|2022-06-05|  2022-06|2022|    6|   5|  18|      5|     30|        1|  

In [23]:
transactions = transactions.withColumn('unix_millis' , sf.unix_millis('TransactionDate'))

In [24]:

transactions = transactions.withColumn('unix_seconds' , sf.unix_seconds('TransactionDate'))
transactions.show()

+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+------------+
|TransactionID|    TransactionDate|Amount|  DateOnly|YearMonth|year|month| day|hour|minutes|seconds|dayofweek|weekofyear|UnixSeconds|  unix_millis|unix_seconds|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+------------+
|         T001|2025-08-28 14:30:00| 120.5|2025-08-28|  2025-08|2025|    8|  28|  14|     30|      0|        5|        35| 1756384200|1756384200000|  1756384200|
|         T002|2024-12-31 23:59:59| 250.0|2024-12-31|  2024-12|2024|   12|  31|  23|     59|     59|        3|         1| 1735685999|1735685999000|  1735685999|
|         T003|2023-01-15 08:15:45|  75.2|2023-01-15|  2023-01|2023|    1|  15|   8|     15|     45|        1|         2| 1673766945|1673766945000|  1673766945|
|         T004|2022-06-05 18:05:30

In [25]:
transactions.printSchema()

root
 |-- TransactionID: string (nullable = true)
 |-- TransactionDate: timestamp (nullable = true)
 |-- Amount: double (nullable = true)
 |-- DateOnly: date (nullable = true)
 |-- YearMonth: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- seconds: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- weekofyear: integer (nullable = true)
 |-- UnixSeconds: long (nullable = true)
 |-- unix_millis: long (nullable = true)
 |-- unix_seconds: long (nullable = true)



In [26]:
# Calcule le nombre de jours écoulés depuis la date de transaction jusqu’à aujourd’hui.
transactions = transactions.withColumn('days_since_transaction' , sf.datediff(sf.current_date() , sf.col('TransactionDate') ))
transactions.show()

+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+------------+----------------------+
|TransactionID|    TransactionDate|Amount|  DateOnly|YearMonth|year|month| day|hour|minutes|seconds|dayofweek|weekofyear|UnixSeconds|  unix_millis|unix_seconds|days_since_transaction|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+------------+----------------------+
|         T001|2025-08-28 14:30:00| 120.5|2025-08-28|  2025-08|2025|    8|  28|  14|     30|      0|        5|        35| 1756384200|1756384200000|  1756384200|                     1|
|         T002|2024-12-31 23:59:59| 250.0|2024-12-31|  2024-12|2024|   12|  31|  23|     59|     59|        3|         1| 1735685999|1735685999000|  1735685999|                   241|
|         T003|2023-01-15 08:15:45|  75.2|2023-01-15|  2023-01|2023|    1|  15| 

In [27]:
# Calcule le nombre de secondes écoulées entre chaque transaction et 2025-01-01 00:00:00.

# Définir la date de référence
ref_date = sf.lit("2025-01-01 00:00:00")

transactions = transactions.withColumn(
    "seconds_diff",
    sf.unix_timestamp(sf.to_timestamp("TransactionDate")) - sf.unix_timestamp(sf.to_timestamp(ref_date))
)

transactions.select("TransactionID", "TransactionDate", "seconds_diff").show(truncate=False)

+-------------+-------------------+------------+
|TransactionID|TransactionDate    |seconds_diff|
+-------------+-------------------+------------+
|T001         |2025-08-28 14:30:00|20698200    |
|T002         |2024-12-31 23:59:59|-1          |
|T003         |2023-01-15 08:15:45|-61919055   |
|T004         |2022-06-05 18:05:30|-81240870   |
|T005         |NULL               |NULL        |
+-------------+-------------------+------------+



In [28]:
"""
Ajoute 7 jours à chaque transaction (TransactionDate + interval 7 days).
Ajoute 2 heures à chaque transaction.
Retire 30 minutes.
Crée une colonne EndOfMonth contenant le dernier jour du mois.
Crée une colonne StartOfQuarter contenant le premier jour du trimestre.
"""

'\nAjoute 7 jours à chaque transaction (TransactionDate + interval 7 days).\nAjoute 2 heures à chaque transaction.\nRetire 30 minutes.\nCrée une colonne EndOfMonth contenant le dernier jour du mois.\nCrée une colonne StartOfQuarter contenant le premier jour du trimestre.\n'

In [29]:
# Ajoute 7 jours à chaque transaction
transactions.show(truncate=False)

+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+------------+----------------------+------------+
|TransactionID|TransactionDate    |Amount|DateOnly  |YearMonth|year|month|day |hour|minutes|seconds|dayofweek|weekofyear|UnixSeconds|unix_millis  |unix_seconds|days_since_transaction|seconds_diff|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+------------+----------------------+------------+
|T001         |2025-08-28 14:30:00|120.5 |2025-08-28|2025-08  |2025|8    |28  |14  |30     |0      |5        |35        |1756384200 |1756384200000|1756384200  |1                     |20698200    |
|T002         |2024-12-31 23:59:59|250.0 |2024-12-31|2024-12  |2024|12   |31  |23  |59     |59     |3        |1         |1735685999 |1735685999000|1735685999  |241                   |-1          |
|T003         |

In [30]:
transactions.withColumn('TransactionDate_pl_7' , sf.date_add(sf.col('TransactionDate'),7) ).show()

+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+------------+----------------------+------------+--------------------+
|TransactionID|    TransactionDate|Amount|  DateOnly|YearMonth|year|month| day|hour|minutes|seconds|dayofweek|weekofyear|UnixSeconds|  unix_millis|unix_seconds|days_since_transaction|seconds_diff|TransactionDate_pl_7|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+------------+----------------------+------------+--------------------+
|         T001|2025-08-28 14:30:00| 120.5|2025-08-28|  2025-08|2025|    8|  28|  14|     30|      0|        5|        35| 1756384200|1756384200000|  1756384200|                     1|    20698200|          2025-09-04|
|         T002|2024-12-31 23:59:59| 250.0|2024-12-31|  2024-12|2024|   12|  31|  23|     59|     59|        3|         1| 173568

In [31]:
transactions = transactions.withColumn('TransactionDate_pl_7' , sf.date_add(sf.col('TransactionDate'),7) )

In [32]:
# Ajoute 2 heures à chaque transaction.
transactions.select('TransactionID' , 'TransactionDate', 'UnixSeconds').show()

+-------------+-------------------+-----------+
|TransactionID|    TransactionDate|UnixSeconds|
+-------------+-------------------+-----------+
|         T001|2025-08-28 14:30:00| 1756384200|
|         T002|2024-12-31 23:59:59| 1735685999|
|         T003|2023-01-15 08:15:45| 1673766945|
|         T004|2022-06-05 18:05:30| 1654445130|
|         T005|               NULL|       NULL|
+-------------+-------------------+-----------+



In [33]:
transactions = transactions.withColumn(
    "TransactionDate_plus_2",
    sf.to_timestamp(sf.from_unixtime(sf.col("UnixSeconds") + 2*3600))
)
transactions.printSchema()
transactions.select("TransactionID", "UnixSeconds", "TransactionDate_plus_2").show(truncate=False)


root
 |-- TransactionID: string (nullable = true)
 |-- TransactionDate: timestamp (nullable = true)
 |-- Amount: double (nullable = true)
 |-- DateOnly: date (nullable = true)
 |-- YearMonth: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- seconds: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- weekofyear: integer (nullable = true)
 |-- UnixSeconds: long (nullable = true)
 |-- unix_millis: long (nullable = true)
 |-- unix_seconds: long (nullable = true)
 |-- days_since_transaction: integer (nullable = true)
 |-- seconds_diff: long (nullable = true)
 |-- TransactionDate_pl_7: date (nullable = true)
 |-- TransactionDate_plus_2: timestamp (nullable = true)

+-------------+-----------+----------------------+
|TransactionID|UnixSeconds|TransactionDate_plus_2|
+-------------+-----------+------------

In [34]:
transactions.show()

+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+------------+----------------------+------------+--------------------+----------------------+
|TransactionID|    TransactionDate|Amount|  DateOnly|YearMonth|year|month| day|hour|minutes|seconds|dayofweek|weekofyear|UnixSeconds|  unix_millis|unix_seconds|days_since_transaction|seconds_diff|TransactionDate_pl_7|TransactionDate_plus_2|
+-------------+-------------------+------+----------+---------+----+-----+----+----+-------+-------+---------+----------+-----------+-------------+------------+----------------------+------------+--------------------+----------------------+
|         T001|2025-08-28 14:30:00| 120.5|2025-08-28|  2025-08|2025|    8|  28|  14|     30|      0|        5|        35| 1756384200|1756384200000|  1756384200|                     1|    20698200|          2025-09-04|   2025-08-28 16:30:00|
|         T002|2024-12-31 23:59:59| 